# Systolic Array

In this notebook, we learn how systolic array works
and implement a 2 x 2 systolic array using PyRTL.

## Introduction

The heart of the TPU is a systolic array,
an architecture drastically different from that of a CPU or a GPU.  
A key feature of systolic arrays is that **little memory accesses are needed**,
making it extremely efficient in computing matrix multiplication.

## MAC Unit

A systolic array consists of a grid of connected processor elements, AKA MAC units.  
MAC stands for Multiply-and-ACcumulate.  
As the name suggests, a MAC unit takes 2 inputs, multiplies them, and accumulates the result.
For the outputs, a MAC unit outputs the accumulated result and the 2 inputs,
passing them on to other MAC units.

In [1]:
import pyrtl as pr

In [2]:
BIT_WIDTH = 3

We use a function to encapsulate a MAC unit.  
The function arguments are inputs, and the return values are outputs.  
The second argument of `pr.Register` is the name we give to a register.  
The operator `<<=` is defined by PyRTL, indicating a wire connection.

In [3]:
def mac_unit(w_in, d_in, uid):
    accum = pr.Register(BIT_WIDTH * 2, f'reg_accum_{uid}')
    w_out = pr.Register(BIT_WIDTH, f'reg_w_out_{uid}')
    d_out = pr.Register(BIT_WIDTH, f'reg_d_out_{uid}')

    accum.next <<= accum + w_in * d_in
    w_out.next <<= w_in
    d_out.next <<= d_in

    return w_out, d_out, accum

## How A Systolic Array Works within A Row

Recall that a systolic array is a **grid** of processor elements (PEs).
Here we explain how a row of PEs work together.

A PE in a row of a systolic array
- Recieves 2 inputs: `weight` and `data`.
- Performs calculation and outputs `accum`.
- Propagates `weight` to the next PE, i.e. the PE on its right.
- Propagates `data`, more on that in the next section.

One thing to note about the implementation is that  
**We describe the connections sequentially, but the hardware runs in parallel.**  
For example, the first PE passes its weight to the second PE
while the second PE may be performing multiplication.

In [4]:
def systolic_row(weight, data, row):
    d_outs = []
    accums = []

    for col, d_in in enumerate(data):
        weight, d_out, accum = mac_unit(weight, d_in, f'{row}{col}')
        d_outs.append(d_out)
        accums.append(accum)

    return d_outs, accums

## How A Systolic Array Works Between Rows

Similar to `weight`, each row propagates its row of `data` signals to the next row.  
In the function `systolic_array`, we connect each `weight` with a row of PEs.  
`outs` is the list of accumulated outputs of all the PEs.

In [5]:
def systolic_array(weights, data):
    outs = []
    for row, weight in enumerate(weights):
        data, accums = systolic_row(weight, data, row)
        for accum in accums:
            outs.append(accum)
    return outs

## I/O Setups

Below are setups for the simulation.
`weights` and `data` are where elements of the 2 matrix would go,
and `outs` would be the output.

In [6]:
ARRAY_DIM = 2
weights = [pr.Input(BIT_WIDTH, f'in_w_{idx}') for idx in range(ARRAY_DIM)]
data = [pr.Input(BIT_WIDTH, f'in_d_{idx}') for idx in range(ARRAY_DIM)]
outs = [pr.Output(BIT_WIDTH * 2, f'out_{idx:02b}') for idx in range(ARRAY_DIM**2)]

In [7]:
out_vals = systolic_array(weights, data)
for out, out_val in zip(outs, out_vals):
    out <<= out_val

In [8]:
sim_trace = pr.SimulationTrace()
sim = pr.Simulation(tracer=sim_trace)

## How A Systolic Array Works

Suppose we are doing this matrix multiplication:
$$\mathcal{W} \cdot \mathcal{D}=
\begin{bmatrix} 1 & 2 \\ 3 & 4 \end{bmatrix}
\cdot
\begin{bmatrix} 4 & 5 \\ 6 & 7 \end{bmatrix}=
\begin{bmatrix} 16 & 19 \\ 36 & 43 \end{bmatrix} = \mathcal{A}$$

Here is a figure of the systolic array:
```
          in_d_0      in_d_1
          |           |
          V           V
in_w_0 -> PE[0, 0] -> PE[0, 1]
          |           |
          V           V
in_w_1 -> PE[1, 0] -> PE[1, 1]
```

Since weights and data are propagated at each cycle, we need to pad inputs to ensure correctness.

In [9]:
PAD = 0

In [10]:
sim.step({'in_w_0': 2, 'in_w_1': PAD, 'in_d_0': 6, 'in_d_1': PAD})

We first input $\mathcal{W}_{0, 1}$ and $\mathcal{D}_{1, 0}$
into the systolic array.  
In this cycle, only `PE[0, 0]` is executing, and the results will show up in the next cycle.

```
       6           PAD
       |           |
       V           V
  2 -> PE[0, 0] -> PE[0, 1]
       |           |
       V           V
PAD -> PE[1, 0] -> PE[1, 1]
```

In [11]:
sim.step({'in_w_0': 1, 'in_w_1': 4, 'in_d_0': 4, 'in_d_1': 7})
print(f'Output of PE[0, 0]: {sim.inspect("out_00")}')

Output of PE[0, 0]: 12


In this step, `PE[0, 1]` and `PE[1, 0]` are put to work
because they received input from `PE[0, 0]`.  
This is why `in_w_1` and `in_d_1` need padding: they are waiting for `PE[0, 0]`.  
`PE[0, 0]` is also doing new calculations at the same time.
In this case, it is accumulating $1 \times 4$ to the output.
```
     4           7
     |           |
     V        2  V
1 -> PE[0, 0] -> PE[0, 1]
     | 6         |
     V           V
4 -> PE[1, 0] -> PE[1, 1]
```

In [12]:
sim.step({'in_w_0': PAD, 'in_w_1': 3, 'in_d_0': PAD, 'in_d_1': 5})
print(
    f'Output of PE[0, 0]: {sim.inspect("out_00")}',
    f'Output of PE[0, 1]: {sim.inspect("out_01")}',
    f'OUtput of PE[1, 0]: {sim.inspect("out_10")}',
    sep='\n'
)

Output of PE[0, 0]: 16
Output of PE[0, 1]: 14
OUtput of PE[1, 0]: 24


In this step, we are pushing the rest of the matrix elements into the systolic array.  
Now that `PE[1, 1]` received inputs, it is put to work.
```
       PAD         5
       |           |
       V        1  V
PAD -> PE[0, 0] -> PE[0, 1]
       | 4         | 7
       V        4  V
  3 -> PE[1, 0] -> PE[1, 1]
```

`PE[0, 0]` has completed its computation. What is the result?  
In the first cycle, it accumulated $\mathcal{W}_{0, 1} \cdot \mathcal{D}_{1, 0}$.  
In the second cycle, it accumulated
$\mathcal{W}_{0, 1} \cdot \mathcal{D}_{1, 0} + \mathcal{W}_{0, 0} \cdot \mathcal{D}_{0, 0}$.  
The result is exactly $\mathcal{A}_{0, 0}$.

In [13]:
sim.step({'in_w_0': PAD, 'in_w_1': PAD, 'in_d_0': PAD, 'in_d_1': PAD})
print(
    f'Output of PE[1, 0]: {sim.inspect("out_10")}',
    f'Output of PE[0, 1]: {sim.inspect("out_01")}',
    f'Output of PE[1, 1]: {sim.inspect("out_11")}',
    sep='\n'
)

Output of PE[1, 0]: 36
Output of PE[0, 1]: 19
Output of PE[1, 1]: 28


In this step, `PE[0, 1]` and `PE[1, 0]` have completed their computations.  
`PE[0, 1]` computed $\mathcal{W}_{0, 1} \cdot \mathcal{D}_{1, 1} +
\mathcal{W}_{0, 0} \cdot \mathcal{D}_{0, 1} = \mathcal{A}_{0, 1}$.  
`PE[1, 0]` computed $\mathcal{W}_{1, 1} \cdot \mathcal{D}_{1, 0} +
\mathcal{W}_{1, 0} \cdot \mathcal{D}_{0, 0} = \mathcal{A}_{1, 0}$.
```
       PAD         PAD
       |           |
       V           V
PAD -> PE[0, 0] -> PE[0, 1]
       |           | 5
       V        3  V
PAD -> PE[1, 0] -> PE[1, 1]
```

In [14]:
sim.step({'in_w_0': PAD, 'in_w_1': PAD, 'in_d_0': PAD, 'in_d_1': PAD})
print(
    f'Output of PE[0, 0]: {sim.inspect("out_00")}',
    f'Output of PE[0, 1]: {sim.inspect("out_01")}',
    f'OUtput of PE[1, 0]: {sim.inspect("out_10")}',
    f'Output of PE[1, 1]: {sim.inspect("out_11")}',
    sep='\n'
)

Output of PE[0, 0]: 16
Output of PE[0, 1]: 19
OUtput of PE[1, 0]: 36
Output of PE[1, 1]: 43


In the final step, `PE[1, 1]` has completed its computation,
and the result is $\mathcal{A}_{1, 1}$.

## Conclusion

Matrix elements are pumped through the systolic array,
just like how the heart pumps blood periodically, hence the name.  
The number of cycles needed to multiply $n \times n$ matrices is $\mathcal{O}(n)$.  
It still fascinates me how everything falls into place perfectly.

Here is the waveform diagram:

In [15]:
sim_trace.render_trace()

<IPython.core.display.Javascript object>

## References

- [PyRTL](https://ucsbarchlab.github.io/PyRTL/)
- [An in-depth look at Google’s first Tensor Processing Unit (TPU)](https://cloud.google.com/blog/products/ai-machine-learning/an-in-depth-look-at-googles-first-tensor-processing-unit-tpu)
- [Systolic-Array Implementation of Matrix-By-Matrix Multiplication](http://ecelabs.njit.edu/ece459/lab3.php)
- [Systolic array - Wikipedia](https://en.wikipedia.org/wiki/Systolic_array)
- [Systolic Array Matrix Multiplication](http://web.cecs.pdx.edu/~mperkows/temp/May22/0020.Matrix-multiplication-systolic.pdf)